In [0]:
!nvidia-smi

Thu Oct 10 07:15:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [15]:
!pip3 uninstall chainer
!pip3 install chainer==1.24.0

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 163kB 47.3MB/s 
  Created wheel for chainer: filename=chainer-1.24.0-cp36-cp36m-linux_x86_64.whl size=4793968 sha256=8b6aaf48b0fd036a2efd15d27fc6012e37d8b300793bac73cea9a7c4f0fb80b0
  Stored in directory: /root/.cache/pip/wheels/95/79/8e/3eb03d4771f898f8063dd202876b9a8669e7cce8e27b90f20f
Successfully built chainer


In [3]:
!apt-get install python3-pip
!pip3 install scipy
!pip3 install h5py
!apt-get install python-h5py
!apt-get install libopenjp2-7-dev
!apt-get install libtiff5
!pip3 install Pillow
!apt-get install espeak
!apt-get install python3-picamera
!apt-get install libatlas-base-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pip python3-pkg-resources python3-secretstorage python3-setuptools
  python3-six python3-wheel python3-xdg
0 upgraded, 15 newly installed, 0 to remove and 8 not upgraded.
Need to get 2,883 kB of archiv

In [4]:
!apt-get install git
!git clone https://github.com/apple2373/chainer-caption.git
%cd chainer-caption

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Cloning into 'chainer-caption'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 363 (delta 10), reused 0 (delta 0), pack-reused 345
Receiving objects: 100% (363/363), 2.99 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (223/223), done.
bash: download.sh: No such file or directory
python3: can't open file 'sample_code_beam.py': [Errno 2] No such file or directory


In [0]:
## After uploading external image im2
!mv /content/im2.jpg /content/chainer-caption/im2.jpg

In [28]:
!bash download.sh
!python3 sample_code_beam.py --rnn-model ./data/caption_en_model40.model --cnn-model ./data/ResNet50.model --vocab ./data/MSCOCO/mscoco_caption_train2014_processed_dic.json --gpu -1 --img ./im2.jpg

<sos> a traffic light sitting on top of a pole <eos>
-8.06228917883709
<sos> a traffic light sitting on top of a street <eos>
-9.018726706970483
<sos> a traffic light sitting on top of a tall building <eos>
-9.246615365613252


In [18]:
# !wget https://raw.githubusercontent.com/yoshihiroo/programming-workshop/master/image_captioning_and_speech/image_captioning.py

--2019-10-10 07:51:25--  https://raw.githubusercontent.com/yoshihiroo/programming-workshop/master/image_captioning_and_speech/image_captioning.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1779 (1.7K) [text/plain]
Saving to: ‘image_captioning.py’

image_captioning.py 100%[===================>]   1.74K  --.-KB/s    in 0s      

2019-10-10 07:51:26 (309 MB/s) - ‘image_captioning.py’ saved [1779/1779]



In [0]:
# !python3 image_captioning.py

In [0]:
##### sample_code_beam.py ######

#!/usr/bin/env python
# -*- coding: utf-8 -*-

'''
Sample code to generate caption using beam search
'''
import sys
import json
import os
# comment out the below if you want to do type check. Remeber this have to be done BEFORE import chainer
# os.environ["CHAINER_TYPE_CHECK"] = "0"
import chainer 

import argparse
import numpy as np
import math
from chainer import cuda
import chainer.functions as F
from chainer import cuda, Function, gradient_check, Variable, optimizers
from chainer import serializers

sys.path.append('./code')
from CaptionGenerator import CaptionGenerator

#Parse arguments
parser = argparse.ArgumentParser()
parser.add_argument("-g", "--gpu",default=-1, type=int, help=u"GPU ID.CPU is -1")
parser.add_argument('--vocab',default='./data/MSCOCO/mscoco_caption_train2014_processed_dic.json', type=str,help='path to the vocaburary json')
parser.add_argument('--img',default='./sample_imgs/dog.jpg', type=str,help='path to the image')
parser.add_argument('--cnn-model', type=str, default='./data/ResNet50.model',help='place of the ResNet model')
parser.add_argument('--rnn-model', type=str, default='./data/caption_model.model',help='place of the caption model')
parser.add_argument('--beam',default=3, type=int,help='beam size in beam search')
parser.add_argument('--depth',default=50, type=int,help='depth limit in beam search')
parser.add_argument('--lang',default="<sos>", type=str,help='special word to indicate the langauge or just <sos>')
args = parser.parse_args()

caption_generator=CaptionGenerator(
    rnn_model_place=args.rnn_model,
    cnn_model_place=args.cnn_model,
    dictonary_place=args.vocab,
    beamsize=args.beam,
    depth_limit=args.depth,
    gpu_id=args.gpu,
    first_word= args.lang,
    )

captions = caption_generator.generate(args.img)
for caption in captions:
    print (" ".join(caption["sentence"]))
    print (caption["log_likelihood"])


In [0]:
##### code/CaptionGenerator.py #######

# -*- coding: utf-8 -*-
#!/usr/bin/env python

'''
caption generation module by beam search
currently you cannot use efficient generation by batch. Batch size is always one 

beam search might have a small bug or not the most efficient, but seems to be ok.
'''

import os
import chainer 

import numpy as np
import json
import math
from copy import deepcopy

from chainer import cuda
import chainer.functions as F
from chainer import cuda, Function, gradient_check, Variable, optimizers
from chainer import serializers

from image_loader import Image_loader
from ResNet50 import ResNet
from Image2CaptionDecoder import Image2CaptionDecoder

#priority queue
#reference: http://www.bogotobogo.com/python/python_PriorityQueue_heapq_Data_Structure.php
try:
    import Queue as Q  # ver. < 3.0
except ImportError:
    import queue as Q

import heapq

class CaptionGenerator(object):
    def __init__(self,rnn_model_place,cnn_model_place,dictonary_place,beamsize=3,depth_limit=50,gpu_id=-1,first_word="<sos>",hidden_dim=512,mean="imagenet"):
        self.gpu_id=gpu_id
        self.beamsize=beamsize
        self.depth_limit=depth_limit
        self.image_loader=Image_loader(mean)
        self.index2token=self.parse_dic(dictonary_place)

        self.cnn_model=ResNet()
        serializers.load_hdf5(cnn_model_place, self.cnn_model)
        self.cnn_model.train = False

        self.rnn_model=Image2CaptionDecoder(len(self.token2index),hidden_dim=hidden_dim)
        if len(rnn_model_place) > 0:
            serializers.load_hdf5(rnn_model_place, self.rnn_model)
        self.rnn_model.train = False

        self.first_word=first_word

        #Gpu Setting
        global xp
        if self.gpu_id >= 0:
            xp = cuda.cupy 
            cuda.get_device(gpu_id).use()
            self.cnn_model.to_gpu()
            self.rnn_model.to_gpu()
        else:
            xp=np

    def parse_dic(self,dictonary_place):
        with open(dictonary_place, 'r') as f:
            json_file = json.load(f)
        if len(json_file) < 10:#this is ad-hock. I need to distinguish new format and old format...
            self.token2index = { word['word']:word['idx'] for word in json_file["words"]}
        else:
            self.token2index = json_file

        return {v:k for k,v in self.token2index.items()}

    def successor(self,current_state):
        '''
        Args:
            current_state: a stete, python tuple (hx,cx,path,cost)
                hidden: hidden states of LSTM
                cell: cell states LSTM
                path: word indicies so far as a python list  e.g. initial is self.token2index["<sos>"]
                cost: negative log likelihood

        Returns:
            k_best_next_states: a python list whose length is the beam size. possible_sentences[i] = {"indicies": list of word indices,"cost":negative log likelihood so far}

        '''

        word=[xp.array([current_state["path"][-1]],dtype=xp.int32)]
        hx=current_state["hidden"]
        cx=current_state["cell"]
        hy, cy, next_words=self.rnn_model(hx,cx,word)

        word_dist=F.softmax(next_words[0]).data[0]#possible next word distributions
        k_best_next_sentences=[]
        for i in range(self.beamsize):
            next_word_idx=int(xp.argmax(word_dist))
            k_best_next_sentences.append(\
                {\
                "hidden":hy,\
                "cell":cy,\
                "path":deepcopy(current_state["path"])+[next_word_idx],\
                "cost":current_state["cost"]-xp.log(word_dist[next_word_idx])
                }\
                )
            word_dist[next_word_idx]=0

        return hy, cy, k_best_next_sentences

    def beam_search(self,initial_state):
        '''
        Beam search is a graph search algorithm! So I use graph search abstraction

        Args:
            initial state: an initial stete, python tuple (hx,cx,path,cost)
            each state has 
                hx: hidden states
                cx: cell states
                path: word indicies so far as a python list  e.g. initial is self.token2index["<sos>"]
                cost: negative log likelihood

        Returns:
            captions sorted by the cost (i.e. negative log llikelihood)
        '''
        found_paths=[]
        top_k_states=[initial_state]
        while (len(found_paths) < self.beamsize):
            #forward one step for all top k states, then only select top k after that
            new_top_k_states=[]
            for state in top_k_states:
                #examine to next five possible states
                hy, cy, k_best_next_states = self.successor(state)
                for next_state in k_best_next_states:
                    new_top_k_states.append(next_state)
            selected_top_k_states=heapq.nsmallest(self.beamsize, new_top_k_states, key=lambda x : x["cost"])

            #within the selected states, let's check if it is terminal or not.
            top_k_states=[]
            for state in selected_top_k_states:
                #is goal state? -> yes, then end the search
                if state["path"][-1] == self.token2index["<eos>"] or len(state["path"])==self.depth_limit:
                    found_paths.append(state)
                else:
                    top_k_states.append(state)

        return sorted(found_paths, key=lambda x: x["cost"]) 

    def beam_search0(self,initial_state):
        #original one. This takes much memory
        '''
        Beam search is a graph search algorithm! So I use graph search abstraction
        Args:
            initial state: an initial stete, python tuple (hx,cx,path,cost)
            each state has 
                hx: hidden states
                cx: cell states
                path: word indicies so far as a python list  e.g. initial is self.token2index["<sos>"]
                cost: negative log likelihood
        Returns:
            captions sorted by the cost (i.e. negative log llikelihood)
        '''
        found_paths=[]
        q = Q.PriorityQueue()
        q.put((0,initial_state))
        while (len(found_paths) < self.beamsize):
            i=0
            # this is just a one step ahead? 
            while not q.empty():
                if i == self.beamsize:
                    break
                state=q.get()[1]
                #is goal state? -> yes, then end the search
                if state["path"][-1] == self.token2index["<eos>"] or len(state["path"])==self.depth_limit:
                    found_paths.append(state)
                    continue
                #examine to next five possible states and add to priority queue 
                hy, cy, k_best_next_states = self.successor(state)
                for next_state in k_best_next_states:
                    q.put((state["cost"],next_state))
                i+=1

        return sorted(found_paths, key=lambda x: x["cost"]) 

    def generate(self,image_file_path):
        '''
        Args:
            image_file_path: image_file_path
        '''
        img=self.image_loader.load(image_file_path)
        return self.generate_from_img(img)


    def generate_from_img_feature(self,image_feature):
        if self.gpu_id >= 0:
            image_feature=cuda.to_gpu(image_feature)

        batch_size=1
        hx=xp.zeros((self.rnn_model.n_layers, batch_size, self.rnn_model.hidden_dim), dtype=xp.float32)
        cx=xp.zeros((self.rnn_model.n_layers, batch_size, self.rnn_model.hidden_dim), dtype=xp.float32)
        
        hy,cy = self.rnn_model.input_cnn_feature(hx,cx,image_feature)


        initial_state={\
                    "hidden":hy,\
                    "cell":cy,\
                    "path":[self.token2index[self.first_word]],\
                    "cost":0,\
                }\

        captions=self.beam_search(initial_state)

        caption_candidates=[]
        for caption in captions:
            sentence= [self.index2token[word_idx] for word_idx in caption["path"]]
            log_likelihood = -float(caption["cost"])#cost is the negative log likelihood
            caption_candidates.append({"sentence":sentence,"log_likelihood":log_likelihood})

        return caption_candidates

    def generate_from_img(self,image_array):
        '''Generate Caption for an Numpy Image array
        
        Args:
            image_array: numpy array of image

        Returns:
            list of generated captions, sorted by the cost (i.e. negative log llikelihood)

            The structure is [caption,caption,caption,...]
            Where caption = {"sentence": a generated sentence as a python list of word, "log_likelihood": The log llikelihood of the generated sentence} 

        '''
        if self.gpu_id >= 0:
            image_array=cuda.to_gpu(image_array)
        image_feature=self.cnn_model(image_array, "feature").data.reshape(1,1,2048)#次元が一つ多いのは、NstepLSTMはsequaenceとみなすから。(sequence size, batch size, feature dim)ということ

        return self.generate_from_img_feature(image_feature)

if __name__ == '__main__':
    xp=np
    #test code on cpu
    caption_generator=CaptionGenerator(
        rnn_model_place="../experiment1/caption_model1.model",\
        cnn_model_place="../data/ResNet50.model",\
        dictonary_place="../data/MSCOCO/mscoco_caption_train2014_processed_dic.json",\
        beamsize=3,depth_limit=50,gpu_id=-1,)

    # batch_size=1
    # hx=xp.zeros((caption_generator.rnn_model.n_layers, batch_size, caption_generator.rnn_model.hidden_dim), dtype=xp.float32)
    # cx=xp.zeros((caption_generator.rnn_model.n_layers, batch_size, caption_generator.rnn_model.hidden_dim), dtype=xp.float32)
    # img=caption_generator.image_loader.load("../sample_imgs/COCO_val2014_000000185546.jpg")
    # image_feature=caption_generator.cnn_model(img, "feature").data.reshape(1,1,2048)#次元が一つ多いのは、NstepLSTMはsequaenceとみなすから。(sequence size, batch size, feature dim)ということ
    
    # hy,cy = caption_generator.rnn_model.input_cnn_feature(hx,cx,image_feature)
    # initial_state={\
    #             "hidden":hy,\
    #             "cell":cy,\
    #             "path":[caption_generator.token2index["<sos>"]],\
    #             "cost":0,\
    #             }\

    # #successor test
    # next_states= caption_generator.successor(initial_state)
    # print next_states

    # print "beam search test"
    # captions= caption_generator.beam_search(initial_state)
    # print captions

    captions = caption_generator.generate("../sample_imgs/COCO_val2014_000000185546.jpg")
    for caption in captions:
        print(caption["sentence"])
        print(caption["log_likelihood"])

